In [3]:
%config PICompleter.greedy=True #TAB键补全代码

In [14]:
## import library
import os,sys
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import ast
from livelossplot import PlotLossesKeras

In [6]:
#### 1)加载数据
df = pd.read_csv('./iiwa_dataset.csv')
train_set = df.head(6000)
test_set = df.tail(df.shape[0]-6000)

train_datas = [];train_labels = []
for rows in train_set.itertuples():
    _labels = ast.literal_eval(rows.Joint)
    _datas = ast.literal_eval(rows.Pose)
    train_datas.append(np.array(_datas).reshape(21,))
    train_labels.append(np.array(_labels).reshape(6,))
    
train_datas = np.array(train_datas)
train_labels = np.array(train_labels)

test_datas=[];test_labels=[]
for rows in test_set.itertuples():
    _labels = ast.literal_eval(rows.Joint)
    _datas = ast.literal_eval(rows.Pose)
    test_datas.append(np.array(_datas).reshape(21,))
    test_labels.append(np.array(_labels).reshape(6,))

test_datas = np.array(test_datas)
test_labels = np.array(test_labels)

In [7]:
print((train_datas.shape))
print((train_datas[0]))

(6000, 21)
[ 0.06940018  0.05065947  0.21813851 -0.09134055  0.07713289  0.35963126
 -0.17634429  0.13377536  0.49195697 -0.18701058  0.26070565  0.46763395
 -0.35311275  0.17358343  0.41526836 -0.4128781  -0.00727802  0.37909341
 -0.47932624 -0.05744847  0.3782185 ]


In [8]:
### 对数据进行归一化
train_datas_norm = tf.keras.utils.normalize(train_datas,axis=1,order=2)
test_datas_norm = tf.keras.utils.normalize(test_datas,axis=1,order=2)
print(train_datas_norm[0])

[ 0.05131843  0.03746048  0.16130398 -0.06754238  0.05703643  0.26593173
 -0.13039896  0.09892108  0.36378086 -0.13828622  0.19278053  0.34579505
 -0.26111158  0.12835742  0.30707296 -0.30530546 -0.00538178  0.28032315
 -0.35444098 -0.04248066  0.27967619]


In [37]:
### 2)搭建网络
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(1,21)),
    keras.layers.Dense(100,activation='tanh'),
    keras.layers.Dense(100,activation='tanh'),
    keras.layers.Dense(100,activation='tanh'),
    keras.layers.Dense(100,activation='tanh'),
    keras.layers.Dense(6,activation=None)
])
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_3 (Flatten)          (None, 21)                0         
_________________________________________________________________
dense_12 (Dense)             (None, 100)               2200      
_________________________________________________________________
dense_13 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_14 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_15 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_16 (Dense)             (None, 6)                 606       
Total params: 33,106
Trainable params: 33,106
Non-trainable params: 0
__________________________________________________

In [38]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
    loss=tf.keras.losses.MeanSquaredError())
model.fit(train_datas_norm, train_labels, epochs=10)

Epoch 1/10
188/188 [==============================] - 1s 1ms/step - loss: 2.1738
Epoch 2/10
188/188 [==============================] - 0s 2ms/step - loss: 2.1497
Epoch 3/10
188/188 [==============================] - 0s 2ms/step - loss: 2.1486
Epoch 4/10
188/188 [==============================] - 0s 1ms/step - loss: 2.1493
Epoch 5/10
188/188 [==============================] - 0s 2ms/step - loss: 2.1525
Epoch 6/10
188/188 [==============================] - 0s 2ms/step - loss: 2.1538
Epoch 7/10
188/188 [==============================] - 0s 2ms/step - loss: 2.1520
Epoch 8/10
188/188 [==============================] - 0s 2ms/step - loss: 2.1543
Epoch 9/10
188/188 [==============================] - 0s 2ms/step - loss: 2.1466
Epoch 10/10
188/188 [==============================] - 0s 1ms/step - loss: 2.1498


In [42]:
### 尝试使用LM算法
os.chdir(".")
lm_dir = "tf-levenberg-marquardt"
os.chdir(lm_dir)
import levenberg_marquardt as lm

FileNotFoundError: [Errno 2] No such file or directory: 'tf-levenberg-marquardt'

In [45]:
model_wrapper = lm.ModelWrapper(tf.keras.models.clone_model(model))
model_wrapper.compile(
    optimizer=tf.keras.optimizers.SGD(learning_rate=1),
    loss=lm.MeanSquaredError())
model_wrapper.fit(train_datas_norm, train_labels, epochs=100)

Epoch 1/100
188/188 [==============================] - 17s 86ms/step - damping_factor: 0.0100 - attempts: 1.0000 - loss: 2.1221
Epoch 2/100
188/188 [==============================] - 15s 80ms/step - damping_factor: 0.0010 - attempts: 1.0000 - loss: 2.1105
Epoch 3/100
188/188 [==============================] - 16s 86ms/step - damping_factor: 0.0100 - attempts: 1.0000 - loss: 2.0964
Epoch 4/100
188/188 [==============================] - 16s 85ms/step - damping_factor: 1.0000e-05 - attempts: 2.0000 - loss: 2.0886 0s - damping_factor: 1.0000e-05 - attempts: 2.0000 - los
Epoch 5/100
188/188 [==============================] - 16s 83ms/step - damping_factor: 1.0000e-04 - attempts: 3.0000 - loss: 2.0909
Epoch 6/100
188/188 [==============================] - 15s 81ms/step - damping_factor: 1.0000e-04 - attempts: 2.0000 - loss: 2.1048
Epoch 7/100
188/188 [==============================] - 15s 80ms/step - damping_factor: 1.0000e-04 - attempts: 2.0000 - loss: 2.0866
Epoch 8/100
188/188 [==========

188/188 [==============================] - ETA: 0s - damping_factor: 1.0000e-04 - attempts: 2.0000 - loss: 2.103 - 15s 81ms/step - damping_factor: 1.0000e-04 - attempts: 2.0000 - loss: 2.1032
Epoch 54/100
188/188 [==============================] - 15s 77ms/step - damping_factor: 1.0000e-04 - attempts: 2.0000 - loss: 2.1004
Epoch 55/100
188/188 [==============================] - 15s 80ms/step - damping_factor: 1.0000e-05 - attempts: 1.0000 - loss: 2.1083 6 - ETA: 1s - damping_factor: 1.0000e-04 - attempts: 2.0000 
Epoch 56/100
188/188 [==============================] - 15s 78ms/step - damping_factor: 1.0000e-04 - attempts: 2.0000 - loss: 2.1103
Epoch 57/100
188/188 [==============================] - 15s 80ms/step - damping_factor: 1.0000e-04 - attempts: 3.0000 - loss: 2.0908
Epoch 58/100
188/188 [==============================] - 15s 80ms/step - damping_factor: 1.0000e-04 - attempts: 2.0000 - loss: 2.1019
Epoch 59/100
188/188 [==============================] - 16s 84ms/step - damping_fa